# Оценка рисков сердечно-сосудистх заболеваний с использованием AI(TensorFlow)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
data = pd.read_csv('heart.csv')
data.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [ ]:
# Разделение на признаки и целевую переменную
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Определение числовых и категориальных признаков
numeric_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
categorical_features = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [ ]:
# Препроцессинг данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Применение преобразований
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
%pip install tensorflow

In [ ]:
# Построение нейронной сети:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Определение архитектуры сети:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция модели:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

# Обучение модели с ранней остановкой:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - AUC: 0.5515 - accuracy: 0.5266 - loss: 0.6839 - val_AUC: 0.8371 - val_accuracy: 0.7687 - val_loss: 0.5975
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - AUC: 0.8674 - accuracy: 0.7686 - loss: 0.5504 - val_AUC: 0.8513 - val_accuracy: 0.8163 - val_loss: 0.4825
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - AUC: 0.8981 - accuracy: 0.8152 - loss: 0.4219 - val_AUC: 0.8653 - val_accuracy: 0.8163 - val_loss: 0.4520
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - AUC: 0.9299 - accuracy: 0.8590 - loss: 0.3437 - val_AUC: 0.8781 - val_accuracy: 0.8163 - val_loss: 0.4381
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - AUC: 0.9351 - accuracy: 0.8771 - loss: 0.3246 - val_AUC: 0.8836 - val_accuracy: 0.8163 - val_loss: 0.4269
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - AUC: 0.9357 - accuracy: 0.8820 - loss: 0.3224 - val_AUC: 0.8895 - val_accuracy: 0.8299 - val_loss: 0.4155
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0

In [ ]:
# Оценка модели :
# Оценка на тестовых данных
test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}, Test AUC: {test_auc:.4f}")

# Прогноз для нового пациента
def predict_risk(new_patient_data):
    """
    new_patient_data: словарь с признаками пациента
    Возвращает вероятность сердечно-сосудистого заболевания
    """
    df = pd.DataFrame([new_patient_data])
    processed = preprocessor.transform(df)
    return model.predict(processed)[0][0]

# Пример использования на пациенте N (значения взяты произвольне):
patient = {
    'Age': 58,
    'Sex': 'M',
    'ChestPainType': 'ATA',
    'RestingBP': 140,
    'Cholesterol': 289,
    'FastingBS': 0,
    'RestingECG': 'Normal',
    'MaxHR': 172,
    'ExerciseAngina': 'N',
    'Oldpeak': 0.0,
    'ST_Slope': 'Up'
}

risk = predict_risk(patient)
print(f"Risk of heart disease: {risk*100:.2f}%")

Test Accuracy: 0.8859, Test AUC: 0.9305
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Risk of heart disease: 7.41%


## Интерпретация результатов
- Точность (Accuracy): Доля правильных прогнозов
- AUC (Area Under Curve): Способность различать классы (1.0 - идеально)
- Прогноз риска: Вероятность от 0 (низкий риск) до 1 (высокий риск)

### Ключевые особенности решения:
1. Автоматическая предобработка данных:
   - Стандартизация числовых признаков
   - One-Hot кодирование категориальных переменных
2. Архитектура сети:
   - 3 скрытых слоя с регуляризацией Dropout
   - Выходной слой с сигмоидой для бинарной классификации
3. Оптимизация обучения:
   - Ранняя остановка для предотвращения переобучения
   - Адаптивная скорость обучения (Adam)
4. Метрики качества:
   - Accuracy для общей точности
   - AUC для оценки разделительной способности
